### 🛢️ Objetivo del proyecto

Encontrar **la mejor región** para abrir **200 nuevos pozos** de petróleo:

- Predecimos el volumen de reservas (`product`) con **regresión lineal**.
- En cada región:
  - Exploramos 500 puntos.
  - Elegimos los **200 pozos con mayor valor predicho**.
- Calculamos:
  - Beneficio esperado.
  - Riesgo de pérdidas usando **bootstrapping**.
- Condición final:
  - Elegir una región con **riesgo de pérdidas < 2.5 %** y **mayor beneficio promedio**.


### 1. 📦 Importación de librerías

In [1]:
# 1.1 Librerías necesarias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from pathlib import Path
# 1.2 Constate aleatroria
RANDOM_STATE = 54321 

In [2]:
# 1.3 Constantes de negocio

BUDGET = 100_000_000            #  → Presupuesto total (USD).
N_WELLS = 200                   #  → Pozos que se van a desarrollar.
N_POINTS_REGION = 500           #  → Puntos explorados por región.
REVENUE_PER_UNIT = 4_500        #  → Ingreso por unidad de `product` (mil barriles).
REVENUE_PER_BARREL= 4.5         #  → Ingreso por Barril en USD
MIN_UNITS_PER_WELL = 111.1      #  → Producción mínima (en unidades de product) para que un pozo no genere pérdidas
MIN_REVENUE_PER_WELL = 500_000  #  → Ingreso mínimo (500,000 USD). para que un pozo no genere pérdidas

### 2. 📂 Carga y estructura de los datos

Se usan tres archivos:

- `geo_data_0.csv`
- `geo_data_1.csv`
- `geo_data_2.csv`

Cada dataset tiene:

- **100,000 filas** y **5 columnas**: `id`, `f0`, `f1`, `f2`, `product`.
- No hay valores nulos.
- Tipos correctos:
  - `id` → `object`
  - `f0`, `f1`, `f2`, `product` → `float64`




In [3]:

# 2.1 Rutas de Archivos
BASE_DIR = Path.cwd()

DATASET_0 = BASE_DIR / "geo_data_0.csv"
DATASET_1 = BASE_DIR / "geo_data_1.csv"
DATASET_2 = BASE_DIR / "geo_data_2.csv"


# 2.2 Carga de datasets

data_0 = pd.read_csv(DATASET_0)
data_1 = pd.read_csv(DATASET_1)
data_2 = pd.read_csv(DATASET_2)

# En caso de necesitarlo el data set compelto
data = pd.concat([data_0,data_1,data_2])

print(data_0.info())
print(data_1.info())
print(data_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  

Conclusión:  
Los datos están **limpios y listos** para entrenar el modelo sin procesamiento extra.

### 3. 📊 Media de reservas por región

Se calculó la media de `product` para cada región:

- Región `geo_data_0` → **92.5**
- Región `geo_data_1` → **68.825**
- Región `geo_data_2` → **95.0**

Condición de negocio:

- Para no tener pérdidas, cada pozo debe generar **≈ 500,000 USD**.
- Eso equivale a **≈ 111.1 unidades** de `product`.





In [4]:
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
print(data_0.describe())

print(data_1.describe())

print(data_2.describe())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%       

In [6]:
def media_products (data_region, nombre_region):
    mean_data = data_region["product"].mean()
    print(f"\nLa media de la region \"{nombre_region}\" es:", mean_data)

media_products(data_0, "geo_data_0")
media_products(data_1, "geo_data_1")
media_products(data_2, "geo_data_2")


La media de la region "geo_data_0" es: 92.5

La media de la region "geo_data_1" es: 68.825

La media de la region "geo_data_2" es: 95.0


Comparación:

Todas las regiones estan por debajo de  por debajo de `MIN_UNITS_PER_WELL` = 111.1, así que:
- Conclusión: **No basta con ver la media**.  
- Es necesario seleccionar solo los **mejores 200 pozos** de cada región usando el modelo.
- Si tuviera que elegir eligiria la región `geo_data_2`.

Esto quiere decir que si perforaras pozos al azar, esperarías perder dinero.


### 4. 🤖 Modelo de regresión lineal por región

Para cada región se hizo:

1. **Features**: `f0`, `f1`, `f2`  
   **Target**: `product`.
2. División de datos:
   - 75 % entrenamiento
   - 25 % validación
3. Entrenamiento de un **modelo de regresión lineal**.
4. Predicción sobre el conjunto de validación.
5. Cálculo de métricas:
   - Media de `product` predicho.
   - RMSE del modelo.
   - (Extra) R².
   - Longitud (Para comprobación)



In [7]:
# #### 🎯 Definición de features y target Para cada región:


def evaluar_region(data_region, nombre_region):
    
# Quitamos las variables qu podrian introducir ruido y quitamos la variable objetivo
    features = data_region.drop(["product", "id"], axis=1)
# DEjamos solo nuestra variable objetivo
    target = data_region["product"]

# Entrenamos nuestro modelo con 75 % entrenamiento y 25 % validación
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, 
        target, 
        train_size=0.75, 
        random_state=RANDOM_STATE
    )

# Seleccionamos nuestro modelo Regreción Lineal
    model = LinearRegression()

# Entrenamos el modelo
    model.fit(features_train, target_train)
# Predecimos los valores de nuestro conjutno de validación del 25%
    pred_array  = model.predict(features_valid)
# Guardamos los valores predichos en la variale "pred" volvemos el array a DataFrame y conservamos los indices 
    pred = pd.Series(pred_array, index=target_valid.index, name="pred")
# Calculamos el Error medio cudratico para sacarl el RMSE
    mse = mean_squared_error(target_valid, pred)

    print(f"\n===== Resultados para: {nombre_region} =====")
    print("Media de products por region:", pred.mean())
    print("MSE:", mse)
    print("RMSE:", mse**0.5)
    print("R2 :", r2_score(target_valid, pred))
    print("Len :", len(pred))
    pred = pd.Series(pred)
    return target_valid, pred

# Guaramos los datos predichos y los de validacion "reales" para hacer Bootstraping 
target_val_data_0, pred_data_0 = evaluar_region(data_0, "geo_data_0")
target_val_data_1, pred_data_1 = evaluar_region(data_1, "geo_data_1")
target_val_data_2, pred_data_2 = evaluar_region(data_2, "geo_data_2")




===== Resultados para: geo_data_0 =====
Media de products por region: 92.15921155743655
MSE: 1420.0394599680067
RMSE: 37.68341093860808
R2 : 0.27379379799785397
Len : 25000

===== Resultados para: geo_data_1 =====
Media de products por region: 68.445940931533
MSE: 0.7961671155006478
RMSE: 0.8922819708481439
R2 : 0.99962224015718
Len : 25000

===== Resultados para: geo_data_2 =====
Media de products por region: 94.92229500787454
MSE: 1612.2236913303577
RMSE: 40.15250541784856
R2 : 0.20227587794782487
Len : 25000


####  Calidad del modelo por Región


El modelo muestra un desempeño muy desigual entre las tres regiones:

- **`geo_data_0`**
  - Media de productos por región: ~92
  - RMSE ≈ 37.7, relativamente alto frente a la media
  - R² ≈ 0.27 → el modelo explica poca variabilidad  
  - Las predicciones **no son muy fiables** para esta región.

- **`geo_data_1`**
  - Media de productos por región: ~68
  - RMSE ≈ 0.89, error muy bajo
  - R² ≈ 0.9996 → el modelo explica prácticamente toda la variabilidad  
  - Aquí el modelo es **muy preciso y confiable**.

- **`geo_data_2`**
  - Media de productos por región: ~95
  - RMSE ≈ 40.2, también alto frente a la media
  - R² ≈ 0.20 → el modelo explica aún menos variabilidad que en `geo_data_0`  
  - El ajuste es **pobre** y las predicciones son poco útiles.

 En este proyecto usamos RMSE como métrica principal porque castiga mucho más los errores grandes, y para OilyGiant es especialmente peligroso sobreestimar el volumen de reservas en algunos pozos.

 Si tuviera que elegir una Región con los los resultados obtenidos eligiria `geo_data_1` por que tiene el menor RMSE y un modelo muy preciso.



In [8]:
# Revisamos que los datos esten correctos.
pred_data_0.head()

47590    110.101190
39469    122.700895
88291     84.023603
46565     95.975692
55316     83.437307
Name: pred, dtype: float64

El proceso se encapsuló en funciones para **evitar código duplicado** y repetirlo en las tres regiones.


### 💵 Parámetros de negocio y fórmula de beneficio

Parámetros usados:

- Presupuesto total: **100,000,000 USD**
- Pozos a desarrollar: **200**
- Ingreso por unidad de `product`: **4500 USD**


La idea es:

- Ordenar pozos por **predicción del modelo**.
- Tomar los **200 mejores**.
- Usar su `product` real para calcular la **ganancia total**.


### 6. 🎲 Bootstrapping: simulación de ganancias y riesgos

Con la función `ingreso_region` se hizo lo siguiente para cada región:

1. Se toman las **predicciones** y los valores reales (`product`) del conjunto de validación.
2. Se ejecutan **1000 simulaciones**:
   - En cada simulación:
     - Se eligen **500 pozos** con reemplazo (bootstrap).
     - Se ordenan por la predicción del modelo.
     - Se seleccionan los **200 mejores**.
     - Se calcula el **beneficio** con la fórmula de negocio.
3. De los 1000 valores de beneficio se obtienen:
   - Beneficio promedio.
   - Intervalo de confianza al 95 % (cuantiles 2.5 % y 97.5 %).
   - Probabilidad de pérdidas (beneficio < 0).
   - ROI promedio y ROI efectivo.

Esto permite medir **rentabilidad** y **riesgo** de cada región.


In [9]:
def ingreso_region(target_val_data, pred_data_region, name):
# Se re asigan las variables para poder leer mas facil el modelo.
    target = target_val_data
    probabilities = pred_data_region

# Calculamos el ingreso de nuestro Bootstraping 
    def revenue(target_subsample, probs_subsample, count):
        probs_sorted = probs_subsample.sort_values(ascending=False)
        # Usamos los datos reales para saber cuanto realmente ganariamos, y no cuanto precide el modelo que ganariamos.
        selected = target_subsample[probs_sorted.index].iloc[:count]
        ingreso_total = REVENUE_PER_UNIT * selected.sum()
        # Restamos el presupuesto de ($500,000 DLS)
        return ingreso_total - BUDGET

# Definimos nuestro modelo de Aleatoriedad  con RAMDOM_STATE para poder hacer reproducible el ejecicio.
    state = np.random.RandomState(RANDOM_STATE)

# Creamos nuestro Bootstraping
    values = []
    for _ in range(1000):
        target_subsample = target.sample(
            # Usamos N_POINTS_REGION para definir los 200 pozos que queremos de los 500 puntos = N_WELLS.
            n=N_POINTS_REGION,
            replace=True,
            random_state=state
        )
        # Unimos por el indice las 2 variables
        probs_subsample = probabilities[target_subsample.index]
        # Guardamos los valores de nuestro Bootstraping 
        values.append(revenue(target_subsample, probs_subsample, N_WELLS))

    values = pd.Series(values)
    # Limite inferior
    lower = values.quantile(0.025)
    # limite superior
    upper = values.quantile(0.975)
    # Probabilidad de perder dinero por pozo.
    prob_perdidas = (values < 0).mean()
    # Beneficio promedio posible por  los 200 pozoz seleccionados con el modelo. 
    mean = values.mean()
    # Retorno Sobre la invercion (Cuanto dinero retornamos por cada peso invertivo en porcentaje)
    roi_mean = mean / BUDGET
    # ROI efectivo para ajustarlo al riesgo.
    roi_efectivo = roi_mean * (1 - prob_perdidas)

    print(f"\n===== Resultados para: {name} =====")
    print("Beneficio Promedio :", mean.round(2))
    print("Cuantil del 2.5 % :", lower.round(2))
    print("Cuantil del 97.5 %:", upper.round(2))
    print(f"Probabilidad de pérdidas del proyecto: {prob_perdidas*100:.2f} %")
    print(f"ROI promedio: {roi_mean*100:.2f} %")
    print(f"ROI efectivo: {roi_efectivo*100:.2f} %")

# Retornamos los valores para hacer nuestra tabla.
    return {
        "region": name,
        "beneficio_promedio": mean,
        "q_2_5": lower,
        "q_97_5": upper,
        "prob_perdidas": prob_perdidas,
        "roi_promedio": roi_mean,
        "roi_efectivo": roi_efectivo,
    }


# Ejecutamos la simulación para cada región y guaramos los datos en sus variables.
res_0 = ingreso_region(target_val_data_0, pred_data_0, "geo_data_0")
res_1 = ingreso_region(target_val_data_1, pred_data_1, "geo_data_1")
res_2 = ingreso_region(target_val_data_2, pred_data_2, "geo_data_2")


===== Resultados para: geo_data_0 =====
Beneficio Promedio : 4089561.93
Cuantil del 2.5 % : -963570.86
Cuantil del 97.5 %: 9616291.73
Probabilidad de pérdidas del proyecto: 5.20 %
ROI promedio: 4.09 %
ROI efectivo: 3.88 %

===== Resultados para: geo_data_1 =====
Beneficio Promedio : 4714887.65
Cuantil del 2.5 % : 523824.95
Cuantil del 97.5 %: 9114850.35
Probabilidad de pérdidas del proyecto: 1.20 %
ROI promedio: 4.71 %
ROI efectivo: 4.66 %

===== Resultados para: geo_data_2 =====
Beneficio Promedio : 4209020.14
Cuantil del 2.5 % : -1403231.64
Cuantil del 97.5 %: 9343215.56
Probabilidad de pérdidas del proyecto: 7.40 %
ROI promedio: 4.21 %
ROI efectivo: 3.90 %


### 7.  📉 ¿Por qué usar ROI efectivo y cómo se calcula?

- **ROI promedio** nos dice qué tan rentable es el proyecto en promedio:

  ROI_promedio = beneficio_promedio / presupuesto

- Pero el ROI promedio **no toma en cuenta el riesgo de perder dinero**.

Por eso usamos el **ROI efectivo**, que ajusta el ROI por la probabilidad de pérdidas:

  ROI_efectivo = ROI_promedio * (1 - probabilidad_de_pérdidas)

En pocas palabras: el **ROI efectivo** es el retorno esperado **ya considerando** que en algunos escenarios el proyecto puede terminar en pérdidas.


In [10]:


# Construir tabla con resultados crudos
tabla_resultados = pd.DataFrame([res_0, res_1, res_2]).set_index("region")

# Versión formateada para mostrar
tabla_mostrar = tabla_resultados.copy()

# Renombrar columnas a nombres más claros
tabla_mostrar = tabla_mostrar.rename(columns={
    "beneficio_promedio": "Beneficio esperado (USD)",
    "q_2_5": "Límite inferior 95% (USD)",
    "q_97_5": "Límite superior 95% (USD)",
})

# Redondear montos en dólares
cols_montos = [
    "Beneficio esperado (USD)",
    "Límite inferior 95% (USD)",
    "Límite superior 95% (USD)",
]
tabla_mostrar[cols_montos] = tabla_mostrar[cols_montos].round(2)

# Formato moneda para qeu sea mas facil interpretar los resultados
for col in cols_montos:
    tabla_mostrar[col] = "$" + tabla_mostrar[col].map("{:,.2f}".format)

# Probabilidades y ROI en porcentaje
tabla_mostrar["Prob. de pérdidas (%)"] = (tabla_mostrar["prob_perdidas"] * 100).round(2)
tabla_mostrar["ROI promedio (%)"]      = (tabla_mostrar["roi_promedio"] * 100).round(2)
tabla_mostrar["ROI efectivo (%)"]      = (tabla_mostrar["roi_efectivo"] * 100).round(2)

# Quitar columnas en decimales si ya no se necesitan
tabla_mostrar = tabla_mostrar.drop(columns=["prob_perdidas", "roi_promedio", "roi_efectivo"])

# Mostrar tabla final
tabla_mostrar

,Beneficio esperado (USD),Límite inferior 95% (USD),Límite superior 95% (USD),Prob. de pérdidas (%),ROI promedio (%),ROI efectivo (%)
region,,,,,,
geo_data_0,"$4,089,561.93","$-963,570.86","$9,616,291.73",5.2,4.09,3.88
geo_data_1,"$4,714,887.65","$523,824.95","$9,114,850.35",1.2,4.71,4.66
geo_data_2,"$4,209,020.14","$-1,403,231.64","$9,343,215.56",7.4,4.21,3.90



En este proyecto combinamos el **beneficio promedio**, la **probabilidad de pérdidas** y el **ROI efectivo**, porque no basta con saber cuánto podríamos ganar en promedio: para OilyGiant también es clave limitar los escenarios en los que el proyecto termina en números rojos.

- **geo_data_0**  
  Genera una ganancia promedio bueno (~4.1 M USD), pero con una probabilidad de pérdidas del 5.2 %. Es una región que puede ser rentable, aunque con un nivel de riesgo alto.

- **geo_data_1**  
  Es la región más atractiva: tiene el **mayor beneficio promedio** (~4.7 M USD), la **probabilidad de pérdidas más baja** (1.2 %) y el **mejor ROI efectivo**. En la práctica, esto significa que no solo promete más ganancias, sino que además es la región donde es más probable que ese beneficio realmente se obtenga.

- **geo_data_2**  
  Su beneficio promedio (~4.2 M USD) es similar al de geo_data_0, pero con la **probabilidad de pérdidas más alta** (7.4 %) y pérdidas potenciales mayores. Desde el punto de vista del negocio, es la opción más arriesgada.




### 7. ✅ Elección de la mejor región

Condiciones de negocio:

- Solo se aceptan regiones con **riesgo de pérdidas < 2.5 %**.
- De esas, elegimos la que tiene **mayor beneficio promedio** y mejor ROI.

Interpretación final:

- `geo_data_0` y `geo_data_2` tienen **riesgo de pérdidas > 2.5 %** → se descartan.
- `geo_data_1`:
  - Beneficio esperado más alto (**≈ 4.7 millones USD**).
  - Riesgo de pérdidas **bajo (1.2 %)**.
  - Mejor ROI promedio y ROI efectivo entre las regiones analizadas.

En este proyecto usamos el **ROI efectivo** como criterio final porque combina la rentabilidad esperada con el riesgo de pérdidas, y para OilyGiant no solo importa cuánto podría ganar una región, sino qué tan probable es que ese beneficio se obtenga sin que el proyecto termine en números rojos.

📌 **Conclusión:**  
La región recomendada para abrir los 200 nuevos pozos es **`geo_data_1`**, porque ofrece el **mejor equilibrio entre beneficio esperado y riesgo de pérdidas**, de acuerdo con las condiciones del negocio y el modelo entrenado.

Mi respuesta se mantiene del punto 4.3 pero de manera mucho mas fundamentada e informada.
